In [1]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def divide_into_sectors(n, k):
    assert (isinstance(n, tuple) and len(n) == 2) or isinstance(n, int)

    a, b = 0, 0
    if isinstance(n, int):
        a, b = n, n
    else:
        a, b = n

    sectors = np.zeros((a, b), dtype=int)

    center_x = b // 2
    center_y = a // 2

    for x in range(b):
        for y in range(a):
            angle = np.degrees(-np.arctan2(y - center_y, x - center_x)) % 360
            sector_index = int(angle // (360 / k))
            sectors[y, x] = sector_index 

    return sectors

print(divide_into_sectors(16, 4))

[[1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0]
 [2 2 2 2 2 2 2 2 0 0 0 0 0 0 0 0]
 [2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3]
 [2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3]
 [2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3]
 [2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3]
 [2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3]
 [2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3]
 [2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3]]


In [ ]:
def gen_piechart(file, output_dir='wframes', split=10, sectors=8, figsize=(14.4, 10.8)):
    testimg = mpimg.imread(file)
    # print(testimg)

    fig, ax = plt.subplots(testimg.shape[0] // split, testimg.shape[1] // split)
    fig.set_figwidth(figsize[0])
    fig.set_figheight(figsize[1])
    fig.subplots_adjust(0, 0, 1, 1)
    fig.set_facecolor('black')

    sectors_split = divide_into_sectors(split, sectors)
    for y in range(0, testimg.shape[0] - (testimg.shape[0] % split), split):
        for x in range(0, testimg.shape[1] - (testimg.shape[1] % split), split):
            colors = [[] for _ in range(sectors)]
            # print(colors)
            for i in range(split):
                for j in range(split):
                    colors[sectors_split[i, j]].append(testimg[y + i, x + j])
                
            # print(colors)
            for i in range(sectors):
                cnum = len(colors[i])
                colors[i] = sum([c / cnum for c in colors[i]])
                colors[i] = colors[i].round() / 255
                
            # print(colors)
            ax[y // split, x // split].pie([1] * sectors, colors=colors, 
                                            radius=1.54, explode=[-0.03] * sectors)
            
                
    plt.savefig(output_dir + '/' +  (file if '/' not in file else file.rsplit('/', 1)[1]))
    # plt.show()
    plt.close()

# gen_piechart("", 'wframes', 42, 12, (19.2, 10.8))

In [ ]:
import os
from joblib import Parallel, delayed


if not os.path.exists('wframes'):
    os.mkdir('wframes')
    
Parallel(n_jobs=-1)(delayed(gen_piechart)('frames/' + frame, 'wframes', 20, 8, (14.4, 10.8))
                     for frame in os.listdir('frames')[1:])